In [ ]:
import torch
import json
import os
import sys

sys.path.append(os.getcwd())

from training.policy_network.model import Model 

MODEL_PATH = "models/policy_network/CN2_BN2_RLROP.pth"
ONNX_PATH = "models/policy_network/CN2_BN2_RLROP.onnx"
MAPPING_PATH = "models/policy_network/move_mapping.json"

def export():
    print(f"Ładowanie mapowania z {MAPPING_PATH}...")
    with open(MAPPING_PATH, "r") as f:
        mapping = json.load(f)
        output_size = len(mapping)

    print(f"Inicjalizacja modelu (output size: {output_size})...")
    model = Model(output_size)
    
    model.load_state_dict(torch.load(MODEL_PATH, map_location='cpu'))
    model.eval()

    dummy_input = torch.randn(1, 13, 8, 8, requires_grad=True)

    print(f"Eksportowanie do {ONNX_PATH}...")
    torch.onnx.export(model, 
        dummy_input, 
        ONNX_PATH, 
        export_params=True,        
        opset_version=11,          
        do_constant_folding=True,
        input_names=['input'], 
        output_names=['output'],
        dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
    )
    
    print("Wyeksportowano do formatu .onxx")

if __name__ == "__main__":
    export()

Ładowanie mapowania z models/policy_network/move_mapping.json...
Inicjalizacja modelu (output size: 1792)...
Eksportowanie do models/policy_network/CN2_BN2_RLROP.onnx...
Wyeskoprtowano do formatu .onxx


/tmp/ipykernel_218078/1406171760.py:29: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(model,
